In [ ]:
import numpy as np
import matplotlib.pylab as plt
from pylab import *
import sys
from fft_utils import fft_1d, shift_1d


In [ ]:
##########################
# define a few constants #		##fft python random phase
##########################

e0 = np.loadtxt('837nm.dat')		#load the reference pulse, pump @836 nm
e0 = e0.T
e1 = np.sqrt(e0[1])
e1_ph = e0[2]
e = np.array(e1)*np.exp(-1j*(np.array(e1_ph)))		#reforming E-field

original_trace = np.loadtxt('traces/trace11.frg')		#load FROG trace from experiment
N = len(e0[0])		#timesteps
N_max = np.max(e0[0])
N_min = np.min(e0[0])

t = e0[0]-N_min
T0 = N_max-N_min	#total time duration
dt = T0/(N-1)
df = 1.0/T0		#frequency steps
TD = N			#time delay for FROG-trace generation
freq = np.arange(0,1/dt+df,df)*1000
print(T0)

In [ ]:
A = 3.0			#Amplitude of Raman pulse
a = 1.0			#intensity dependent factor ##it is not used in this program, ignore it.
a2 = 2.50*10**-6	#second order phase factor
a3 = 3.0*10**-9		#third order phase factor

f0 = 1.600/1000.0	#Raman center-wavelength
f1 = 0.25/1000.0	#Red-shifted center-wavelength

i0 = 333.0		#unit: fs
tau1 = T0/2+i0-400	#Raman pulse peak time
tau2 = tau1-700		#Red-shifted pulse peak time

In [ ]:
def gaussian_pulse(T,f,A,phase,tau):
    w0 = f*2*np.pi		#central angular frequency of pulse
    temp = -1j*w0*(t-tau) + 1j*phase
    E = A*np.sqrt(np.exp(-4.0*np.log(2.0)/np.log(np.e)*((t-tau)/T)**2))*np.exp(-temp)
    return E

In [ ]:

def xcorr(e0,e,td):		#shift one pulse by time delay of 'td'
    E_delay = e*1.0
    if td<=0:
        E_delay[0:N+td] = e0[-td:]
        E_delay[N+td:] = [0]*(-td)
        #E_delay=np.roll(e0,td)*([1]*(N+td)+[0]*(-td))
    else:
        E_delay[0:td] = [0]*(td)
        E_delay[td:] = e0[0:N-td]
        #E_delay=np.roll(e0,td)*([0]*(td)+[1]*(N-td))
    return  e*E_delay

In [ ]:
def FROG_generator(e,E,TD):
    FROG = np.zeros((TD,N), dtype=np.float64)
    for td in range(-TD//2,TD//2):
        E_corre = xcorr(e,E,td)
       
        temp0 =np.fft.fftshift(np.fft.fft(E_corre))
        FROG[(td+TD//2)-1] = np.abs(temp0)**2
    return FROG.T

In [ ]:
def array_difference(A,B):
    A = A/np.max(A)
    B = B/np.max(B)
    return np.sum(np.square(A-B))/np.sum(np.square(A))

In [ ]:
def FROGs(t1,t2,i0,A,f0,a,a2,a3, f1, tau1, tau2): 
    phase1 = a2*np.power(t-tau1,2)+a3*np.power(t-tau1,3)
    phase2 = a2*(t-tau2)**2+a3*(t-tau2)**3
    E1 = gaussian_pulse(t1,f0,A,phase1,tau1)
    E2 = gaussian_pulse(t2,f1,a,phase2-0.5* math.pi,tau2)
    E = E1+E2
    FROG = FROG_generator(e,E,TD)
    delta = array_difference(FROG, original_trace)
    return FROG, delta

In [ ]:
def FROG_trace_plot(FROG):
    fig,ax = plt.subplots()
    plt.imshow(FROG)
    plt.title("FROG trace")
    plt.grid(True)
    plt.show()

In [ ]:
def grads(T,i0, A, f0,a, a2, a3, f1, tau1, tau2):
    grad=np.zeros(8)
    epsilon=1e-9
    grad[0]=(FROGs(T+epsilon,T+epsilon,i0,A,f0,a,a2,a3,f1, tau1,tau2)[1]-FROGs(T-epsilon,T-epsilon,i0,A,f0,a,a2,a3, f1, tau1,tau2)[1])/(2*epsilon)
    grad[1]=(FROGs(T,T,i0,A+epsilon,f0,a,a2,a3,f1, tau1,tau2)[1]-FROGs(T,T,i0,A-epsilon,f0,a,a2,a3, f1, tau1,tau2)[1])/(2*epsilon)
    grad[2]=(FROGs(T,T,i0,A,f0+epsilon,a,a2,a3,f1, tau1,tau2)[1]-FROGs(T,T,i0,A,f0-epsilon,a,a2,a3, f1, tau1,tau2)[1])/(2*epsilon)
    grad[3]=(FROGs(T,T,i0,A,f0,a,a2+epsilon,a3,f1, tau1,tau2)[1]-FROGs(T,T,i0,A,f0,a,a2-epsilon,a3, f1, tau1,tau2)[1])/(2*epsilon)
    grad[4]=(FROGs(T,T,i0,A,f0,a,a2,a3+epsilon*(10**-6),f1, tau1,tau2)[1]-FROGs(T,T,i0,A,f0,a,a2,a3-epsilon*(10**-6), f1, tau1,tau2)[1])/(2*epsilon*(10**-6))
    grad[5]=(FROGs(T,T,i0,A,f0,a,a2,a3, f1 + epsilon, tau1,tau2)[1]-FROGs(T,T,i0,A,f0,a,a2,a3, f1-epsilon, tau1,tau2)[1])/(2*epsilon)
    grad[6]=(FROGs(T,T,i0,A,f0,a,a2,a3, f1,tau1 + epsilon,tau2)[1]-FROGs(T,T,i0,A,f0,a,a2,a3, f1,tau1-epsilon,tau2)[1])/(2*epsilon)
    grad[7]=(FROGs(T,T,i0,A,f0,a,a2,a3, f1,tau1,epsilon+tau2)[1]-FROGs(T,T,i0,A,f0,a,a2,a3, f1,tau1, tau2-epsilon)[1])/(2*epsilon)
    return grad

In [ ]:
def gradient_d(T, i0, A, f0, a, a2, a3, f1, tau1, tau2):
    learning_rate=np.array([10000,.5, 1e-7, 5e-13, 1e-16, 1e-9, 50000,50000])
    epochs=2000
    cost=[]
    min_delta = 1
    for i in range(epochs):
        if(i==200):
            learning_rate[0]=5000
            learning_rate[3] = 1e-13
            learning_rate[0] = learning_rate[0]*10
        grad = grads(T,i0,A,f0,a,a2,a3 ,f1,tau1, tau2)
        T = T - learning_rate[0]*grad[0]
        A = A - learning_rate[1]*grad[1]
        f0 = f0 - learning_rate[2]*grad[2]
        a2 = a2 - learning_rate[3]*grad[3]
        a3 = a3 - learning_rate[4]*grad[4]
        f1 = f1 - learning_rate[5]*grad[5]
        tau1 = tau1 - learning_rate[6]*grad[6]
        tau2 = tau2 - learning_rate[7]*grad[7]
        if(f1 < 0):
            f1 = f1 + 0.9*learning_rate[5]*grad[5]
            learning_rate[5] /=10
        (FROG, delta) = FROGs(T,T,i0,A,f0,a,a2,a3,f1, tau1, tau2) 
        print(i, delta, T, f1)
        if(delta < min_delta and f1>0 and T>0 and A>0):
            min_delta = delta
            min_par = np.array([T, A, f0, a2, a3, f1, tau1, tau2])
            min_frog = FROG
        cost.append(delta)
    
    #print("delta = ", min_delta,"\nT =", min_par[0], "\nA =" , min_par[1],"\nf0 =",min_par[2],"\na =", 1, "\na2 =", min_par[3],"\na3 =", min_par[4],"\nf1 =", min_par[5],"\ntau1 =",min_par[6],"\ntau2 =", min_par[7])
    #plt.plot(list(range(epochs)), cost, '-r') 
    #plt.xlabel('Epochs') 
    #plt.ylabel('Delta')
    FROG_trace_plot(min_frog)
    return min_par
    

In [ ]:
T = 500
A = 3.0			#Amplitude of Raman pulse
a = 1.0			#intensity dependent factor ##it is not used in this program, ignore it.
a2 = 2.50*10**-6	#second order phase factor
a3 = 3.0*10**-10		#third order phase factor

f0 = 1.600/1000.0	#Raman center-wavelength
f1 = 0.25/1000.0	#Red-shifted center-wavelength

i0 = 333.0		#unit: fs
tau1 = T0/2+i0-400	#Raman pulse peak time
tau2 = tau1-700		#Red-shifted pulse peak time
min_t = 500
min_i = 0
min_j = 0
min_frog = 2
for i in range (500, 1500, 25):
    x = FROGs(T,T,i0,A,f0,a,a2,a3,f1, tau1, tau2)[1]
    if(x < min_frog):
        x = min_frog
        min_t = T
for i in range(0,3200, 50):
    for j in range(0,2500,50):
        x = FROGs(T,T,i0,A,f0,a,a2,a3,f1, i, j)[1]
        if(x < min_frog):
            x = min_frog
            min_i = i
            min_j = j


print(min_t, min_i, min_j)
gradient_d(min_t, i0, A, f0, a, a2, a3, f1, min_i, min_j)

SyntaxError: invalid syntax (<ipython-input-207-71014316c345>, line 1)